# Chapter 12 - Custom Models and Training with Tensorflow

# Problem 1

How would you describe TensorFlow in a short sentence? What are its main features? Can you name other popular Deep Learning libraries?

TensorFlow is a library to build neural networks.

The fundamental design principle are Tensors, around which all elements of the library are built. TensorFlow takes care of the mapping from code in high-level languages (mainly Python and Go) to hardware instructions in C++ for different backends (CPU, GPU, TPU). TF automatically builds the computational graph of any architecture to compute gradients using autodiff.

Other libraries with similar aims are PyTorch, Caffe, CLTK, Theano, MXNet, Chainer.

# Problem 2

Is TensorFlow a drop-in replacement for NumPy? What are the main differences between the two?

TensorFlow includes many similar functions as Numpy, but there are fundmental design differences. Function names are not always the same and some functions do not behave the same. TensorFlow does not perform any type casting and elements are - by default - not mutable.

# Problem 3

Do you get the same result with tf.range(10) and tf.constant(np.arange(10)) ?

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [4]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

Both return a tensor of dimension 10 with the integers from 0 to 9. But numpy uses 64bit integers by default while TensorFlow uses 32bits.

# Problem 4

Can you name six other data structures available in TensorFlow, beyond regular tensors?

* Sparse Tensors
* Arrays of Tensors
* Ragged Tensors
* Queues
* String Tensors
* Sets

# Titanic Survival Prediction

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
train = pd.read_csv('titanic/train.csv')
y_train = train['Survived'].values.astype('int32')
vars = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']
X_train_in = train[vars]

In [3]:
X_train_in.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S


In [5]:
cols_int = X_train_in.select_dtypes(include=int).columns
cols_num = X_train_in.select_dtypes(include=float).columns
cols_str = X_train_in.select_dtypes(include=object).columns

In [6]:
class SelectFirstCharacter(BaseEstimator, TransformerMixin):
    def fit(self, X):
        return self
    def transform(self, X, y=None):
        return X.apply(lambda x: x.astype(str, skipna=True).str[0], axis=1)

In [7]:
pipe_int = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())
pipe_num = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())
pipe_str = make_pipeline(
    make_column_transformer((SelectFirstCharacter(), ['Cabin']), remainder='passthrough'),
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore'))

In [8]:
pipe = make_column_transformer((pipe_int, cols_int),
                              (pipe_num, cols_num),
                              (pipe_str, cols_str))

In [9]:
X_train = pipe.fit_transform(X_train_in).astype('float32')

In [10]:
print(('Your TensorFlow version: {0}').format(tf.__version__))

Your TensorFlow version: 2.1.0


In [98]:
# model architecture
inputs = tf.keras.Input(shape=(X_train.shape[1],))
hidden = tf.keras.layers.Dense(3, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(hidden)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [99]:
model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

In [100]:
X = tf.convert_to_tensor(X_train.todense())
y = tf.convert_to_tensor(y_train)

In [101]:
model.fit(X, y, epochs=50, verbose=0)

In [102]:
pred = model.predict(X)

In [103]:
pred

array([[0.13218145],
       [0.9107012 ],
       [0.5870275 ],
       [0.8574829 ],
       [0.06688192],
       [0.11465244],
       [0.48641104],
       [0.10640959],
       [0.6105503 ],
       [0.90787256],
       [0.57233703],
       [0.6812581 ],
       [0.11059539],
       [0.06003612],
       [0.68199533],
       [0.6219697 ],
       [0.2342616 ],
       [0.18184103],
       [0.5867541 ],
       [0.6956949 ],
       [0.15653162],
       [0.32446328],
       [0.7353321 ],
       [0.40269288],
       [0.3771449 ],
       [0.3540888 ],
       [0.11571238],
       [0.21652979],
       [0.65295506],
       [0.08450207],
       [0.27194092],
       [0.9771773 ],
       [0.65282047],
       [0.0563173 ],
       [0.45044625],
       [0.20347033],
       [0.11571407],
       [0.10697781],
       [0.640754  ],
       [0.7869931 ],
       [0.50067097],
       [0.83276314],
       [0.11598222],
       [0.86369854],
       [0.7192507 ],
       [0.08446252],
       [0.14175065],
       [0.652

In [83]:
# Instantiate a logistic loss function that expects integer targets.
fun_error = tf.keras.losses.binary_crossentropy

In [84]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

In [86]:
steps = X_train.shape[0] * 100

In [87]:
for _ in range(steps):
    with tf.GradientTape() as tape:
        pred = model(X)
        loss = fun_error(y, pred)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

In [88]:
pred = model(X)
loss = fun_error(y, pred)

In [90]:
steps

89100